In [3]:
import numpy as np
import yfinance as yf
from scipy.stats import norm
import pandas as pd
import datetime

def fetch_closing_prices(symbol, start, end):
    """
    Fetches daily closing prices for a given stock symbol.
    """
    data = yf.download(symbol, start=start, end=end)
    return data['Close']

def compute_log_returns(prices):
    """
    Computes daily logarithmic returns from price data.
    """
    return np.log(prices / prices.shift(1)).dropna()

def value_at_risk(position_size, confidence_level, mean_return, volatility):
    """
    Calculates 1-day Value at Risk (VaR) using the normal distribution assumption.
    """
    z_score = norm.ppf(1 - confidence_level)
    return position_size * (mean_return - volatility * z_score)

def value_at_risk_n_days(position_size, confidence_level, mean_return, volatility, days):
    """
    Calculates multi-day VaR by scaling mean and volatility appropriately.
    """
    z_score = norm.ppf(1 - confidence_level)
    return position_size * (mean_return * days - volatility * np.sqrt(days) * z_score)

if __name__ == '__main__':
    # Step 1: Define parameters
    stock_symbol = 'AAPL'
    investment_amount = 500  # Portfolio value at risk
    confidence = 0.99        # 99% confidence level
    forecast_days = 10       # Predicting VaR over 10 days

    # Step 2: Fetch data
    start_date = datetime.datetime(2020, 1, 1)
    end_date = datetime.datetime(2024, 4, 30)
    prices = fetch_closing_prices(stock_symbol, start_date, end_date)

    # Step 3: Calculate returns
    returns = compute_log_returns(prices)

    # Step 4: Estimate parameters (assume normality)
    # Extract the scalar value from the pandas Series
    mean_daily_return = returns.mean().item()
    daily_volatility = returns.std().item()

    # Step 5: Calculate and display VaR
    # Extract the scalar value from the pandas Series
    var = value_at_risk_n_days(investment_amount, confidence, mean_daily_return, daily_volatility, forecast_days)
    print(f"{forecast_days}-Day Value at Risk (VaR) at {int(confidence*100)}% confidence level: ${var:.2f}")

/tmp/ipython-input-3-1421542742.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(symbol, start=start, end=end)
[*********************100%***********************]  1 of 1 completed

10-Day Value at Risk (VaR) at 99% confidence level: $80.00
